In [118]:
import pandas as pd
import os

csv_files = [
    '../runs/nparam/task4_e30_reqloss0.0_vanilla_rsched0/train3/stats_pred.csv',
    # '../runs/nparam/task4_e30_reqloss50.0_product_rsched0/train/stats_pred.csv',
    # '../runs/nparam/task4_e30_reqloss1.0_product_rsched0/train/stats_pred.csv',
    '../runs/nparam/task4_e30_reqloss1.0_yager_rsched0/train/stats_pred.csv',
    # '../runs/nparam/task4_e30_reqloss1.0_lukasiewicz_rsched0/train/stats_pred_pre.csv',
    '../runs/nparam/task4_e30_reqloss1.0_lukasiewicz_rsched0/train/stats_pred.csv',
    # '../runs/nparam/task4_e30_reqloss1.0_minimum_rsched0/train2/stats_pred.csv',
]

dfs = [pd.read_csv(f) for f in csv_files]

# Drop last row
dfs = [df[:-1] for df in dfs]

# Number of rows per file
for i, df in enumerate(dfs):
    print(f"Number of rows in file {csv_files[i]}: {len(df)}")

# Use the videos that are included in all datasets
videos = set(dfs[0]["Video"])
for df in dfs[1:]:
    videos = videos.intersection(set(df["Video"]))

dfs_pruned = [df[df["Video"].isin(videos)] for df in dfs]

# Display top 5 rows
display(dfs_pruned[0].head())

# Number of videos
print(f"Number of videos: {len(videos)}")

Number of rows in file ../runs/nparam/task4_e30_reqloss0.0_vanilla_rsched0/train3/stats_pred.csv: 132
Number of rows in file ../runs/nparam/task4_e30_reqloss1.0_yager_rsched0/train/stats_pred.csv: 151
Number of rows in file ../runs/nparam/task4_e30_reqloss1.0_lukasiewicz_rsched0/train/stats_pred.csv: 99


,Video,Mean mAP,Mean full violation,Mean perbox violation,Total boxes,Total frames,Total violations
0,train_00000,0.418113,0.004604,0.914366,90782,198,83008.0
1,train_00008,0.517559,0.003989,0.746244,474258,199,353912.0
2,train_00009,0.566405,0.004473,0.967463,17580,185,17008.0
3,train_00013,0.560004,0.005186,0.757732,458728,198,347593.0
4,train_00019,0.493205,0.005503,0.983870,191756,198,188663.0


Number of videos: 99


In [122]:
# Calculate mean of "Mean mAP" column
mean_mAPs = [df["Mean mAP"].mean() for df in dfs_pruned]

# Calculate the sum of Total violations column devided by the Total boxes column
mean_violations = [df["Total violations"].sum() / df["Total boxes"].sum() for df in dfs_pruned]

# Calculate the mean of "Mean mAP" column in the unpruned data
mean_mAPs_unpruned = [f"{df["Mean mAP"].mean():.5f} ({len(df)})" for df in dfs]

# Calculate the sum of Total violations column devided by the Total boxes column in the unpruned data
mean_violations_unpruned = [f"{df["Total violations"].sum() / df["Total boxes"].sum():.5f} ({len(df)})" for df in dfs]

# Show this in a table
df = pd.DataFrame({
    f"Mean mAP ({len(videos)})": mean_mAPs,
    "Mean violations": mean_violations,
    "Mean mAP (all)": mean_mAPs_unpruned,
    "Mean violations (all)": mean_violations_unpruned
}, index=[f.split("/")[-3] for f in csv_files])

# Highlight the best value in the Mean mAP column
styled_df = df.style.highlight_max(subset=[f"Mean mAP ({len(videos)})"], axis=0)
# Highlight the lowest value in the Mean violations column
styled_df = styled_df.highlight_min(subset=["Mean violations"], axis=0)

# Display the styled DataFrame
display(styled_df)

,Mean mAP (99),Mean violations,Mean mAP (all),Mean violations (all)
task4_e30_reqloss0.0_vanilla_rsched0,0.361978,0.732578,0.36880 (132),0.73122 (132)
task4_e30_reqloss1.0_yager_rsched0,0.367091,0.729830,0.36920 (151),0.73422 (151)
task4_e30_reqloss1.0_lukasiewicz_rsched0,0.364698,0.760297,0.36470 (99),0.76030 (99)
